In [43]:
import pandas as pd
import numpy as np
import nltk
import re
import ssl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('data/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)

In [6]:
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
voc_size=5000

In [10]:
messages=X.copy()

In [11]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [12]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [13]:
messages.reset_index(inplace=True)

In [17]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/KishaniKandasamy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
#data_preprocessing
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [30]:
onehot_repr=[one_hot(sentence,voc_size)for sentence in corpus] 
onehot_repr

[[1734, 4044, 4754, 3814, 4504, 435, 3937, 3256, 4235, 2679],
 [4305, 2467, 4776, 3437, 83, 2509, 2053],
 [3844, 2425, 3224, 1693],
 [3826, 1897, 2105, 1672, 4030, 3076],
 [3925, 83, 202, 3252, 3413, 2253, 83, 2842, 1881, 4430],
 [4633,
  1896,
  424,
  545,
  1323,
  481,
  945,
  1636,
  3061,
  3880,
  708,
  2986,
  1319,
  2876,
  2053],
 [4813, 3358, 840, 1041, 2589, 1160, 2102, 4745, 4574, 4154, 1350],
 [1095, 1520, 2189, 956, 2883, 3184, 481, 451, 4574, 4154, 1350],
 [4896, 3025, 3594, 165, 1356, 1214, 2855, 321, 481, 4467],
 [3533, 130, 608, 1901, 4573, 3830, 2057, 1308],
 [274, 4771, 1147, 3329, 4, 4778, 4712, 3343, 1989, 2638, 2768],
 [1672, 4566, 4504, 1214, 481, 2883],
 [2798, 3139, 93, 245, 3815, 1790, 2574, 3641, 2314],
 [3649, 3903, 3598, 3306, 804, 3538, 2686, 4574, 4154, 1350],
 [84, 4903, 4089, 7, 4704, 4574, 4154, 1350],
 [2293, 3479, 2145, 3137, 649, 3291, 1860, 1425, 419, 3887],
 [2856, 3808, 2467],
 [2649, 1780, 2719, 1597, 481, 1718, 2462, 2053],
 [331, 3875, 47

In [31]:
sent_length=15
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3256 4235 2679]
 [   0    0    0 ...   83 2509 2053]
 [   0    0    0 ... 2425 3224 1693]
 ...
 [   0    0    0 ... 4574 4154 1350]
 [   0    0    0 ... 3687 3572  591]
 [   0    0    0 ... 1737 4863 2697]]


In [32]:
embedded_docs[0]

array([   0,    0,    0,    0,    0, 1734, 4044, 4754, 3814, 4504,  435,
       3937, 3256, 4235, 2679], dtype=int32)

In [40]:
embedding_vector_features=40

In [62]:
model=Sequential()

#model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

model.add(LSTM(128,input_shape=(embedded_docs.shape),activation='relu',return_sequences=True))

model.add(Dropout(0.2))

model.add(LSTM(128,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(32,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(4,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 15, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 15, 128)           86528     
                                                                 
 dropout_6 (Dropout)         (None, 15, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 32)                4128      
                                                                 
 dropout_8 (Dropout)         (None, 32)               

In [63]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [64]:
X_final.shape,y_final.shape

((18285, 15), (18285,))

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [66]:
#Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 23ms/step - loss: 0.4953 - accuracy: 0.7669 - val_loss: 0.2301 - val_accuracy: 0.9082
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1690 - accuracy: 0.9354 - val_loss: 0.1978 - val_accuracy: 0.9203
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1084 - accuracy: 0.9598 - val_loss: 0.2363 - val_accuracy: 0.9231
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0690 - accuracy: 0.9769 - val_loss: 0.2937 - val_accuracy: 0.9148
Epoch 5/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0429 - accuracy: 0.9852 - val_loss: 0.4221 - val_accuracy: 0.9175
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0278 - accuracy: 0.9912 - val_loss: 0.4258 - val_accuracy: 0.9168
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0245 - accuracy: 0.9928 - val_loss: 0.4250 - val_accuracy: 0.9163

In [67]:
embeddings=model.predict(embedded_docs)

In [68]:
embeddings

array([[5.46317024e-04, 9.99449670e-01, 3.85352814e-06, 2.34920137e-07],
       [1.00000000e+00, 3.21771161e-08, 4.63336644e-14, 1.03741015e-16],
       [5.61264983e-07, 9.99999404e-01, 2.11328857e-10, 3.11928760e-12],
       ...,
       [1.00000000e+00, 1.27934365e-08, 4.78299685e-15, 6.11228818e-18],
       [5.24142742e-01, 4.74707216e-01, 1.06606458e-03, 8.39785134e-05],
       [1.98201888e-05, 9.99980211e-01, 4.98440791e-08, 1.96793604e-09]],
      dtype=float32)